# Tarea 2 (10 puntos). Operaciones con RDDs
- Obtener información de algún origen de datos (propio o de API)
- Convertir el origen de datos a RDD con pySpark
- Realizar alguna operación en el RDD, como estadísticas descriptivas básicas
- Subir la práctica a un repositorio público y etiquetarla claramente


In [1]:
import os
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum
from pyspark import SparkConf, SparkContext




In [2]:

spark = SparkSession.builder.appName("Mi segunda tarea").master("local[*]") .config("spark.driver.memory", "12g").config("spark.executor.cores", "5").getOrCreate()

In [3]:
df = spark.read.csv("mortgage_data.csv", header=True, inferSchema=True)

In [4]:
#data
df.show(5)

+------+---------+--------+--------+-----+-------------+--------------------+------+----+-------+-----+----------+-------+---------+
|SOURCE|FREQUENCY|SERIESID|GEOLEVEL|GEOID|      GEONAME|              MARKET|PERIOD|YEAR|QUARTER|MONTH|SUPPRESSED| VALUE1|   VALUE2|
+------+---------+--------+--------+-----+-------------+--------------------+------+----+-------+-----+----------+-------+---------+
|  NMDB|   Annual|TOT_ORIG|National|  USA|United States|       All Mortgages|  1998|1998|      4|   12|         0|11683.0|1307227.0|
|  NMDB|   Annual|TOT_ORIG|National|  USA|United States|All Mortgages (Ho...|  1998|1998|      4|   12|         0| 5325.0| 571274.0|
|  NMDB|   Annual|TOT_ORIG|National|  USA|United States|All Mortgages (Re...|  1998|1998|      4|   12|         0| 6358.0| 735953.0|
|  NMDB|   Annual|TOT_ORIG|National|  USA|United States|       All Mortgages|  1999|1999|      4|   12|         0| 9497.0|1089302.0|
|  NMDB|   Annual|TOT_ORIG|National|  USA|United States|All Mortgages

In [5]:
rdd_data = df.rdd

In [6]:
# Contar cuántas columnas tienen valores nulos
null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Mostrar el conteo de valores nulos por columna
null_counts.show()

+------+---------+--------+--------+-----+-------+------+------+----+-------+-----+----------+------+------+
|SOURCE|FREQUENCY|SERIESID|GEOLEVEL|GEOID|GEONAME|MARKET|PERIOD|YEAR|QUARTER|MONTH|SUPPRESSED|VALUE1|VALUE2|
+------+---------+--------+--------+-----+-------+------+------+----+-------+-----+----------+------+------+
|     0|        0|       0|       0|    0|      0|     0|     0|   0|      0|    0|         0|134106|228006|
+------+---------+--------+--------+-----+-------+------+------+----+-------+-----+----------+------+------+



In [7]:
spark.sparkContext.setLogLevel("DEBUG")


In [10]:
# Seleccionar y limpiar la columna 'VALUE1' en el RDD
value1_rdd = rdd_data.map(lambda row: row["VALUE1"]).filter(lambda x: x is not None)

# Calcular estadísticas básicas
count = value1_rdd.count()
mean = value1_rdd.mean()
min_value = value1_rdd.min()
max_value = value1_rdd.max()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 8.0 failed 1 times, most recent failure: Lost task 9.0 in stage 8.0 (TID 60) (LAPTOP-9P9BDJAS executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.

In [ ]:
# Imprimir resultados
print(f"Cantidad de registros en VALUE1: {count}")
print(f"Media de VALUE1: {mean}")
print(f"Valor mínimo de VALUE1: {min_value}")
print(f"Valor máximo de VALUE1: {max_value}")


In [ ]:
spark.stop()
